# Analyze results

In [ ]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import plotly.colors
import plotly.io as pio
import plotly.express as px
import statistics
import pickle
from pprint import pprint
from my_utils import *

In [ ]:
def suurballe_node_disjoint(G: nx.Graph, s, t):
    G_split = nx.DiGraph()

    # Step 1: Split only intermediate nodes
    for v in G.nodes:
        if v not in (s, t):
            G_split.add_edge(f"{v}_in", f"{v}_out", weight=0)

    # Step 2: Replace edges appropriately
    for u, v, data in G.edges(data=True):
        w = data['weight']

        def map_in(x):
            return f"{x}_in" if x not in (s, t) else x

        def map_out(x):
            return f"{x}_out" if x not in (s, t) else x

        G_split.add_edge(map_out(u), map_in(v), weight=w)
        G_split.add_edge(map_out(v), map_in(u), weight=w)

    s_mod = s
    t_mod = t

    # Step 3: First shortest path
    dist, paths = nx.single_source_dijkstra(G_split, s_mod, weight='weight')
    if t_mod not in paths:
        raise nx.NetworkXNoPath(f"No path from {s} to {t}")
    path1 = paths[t_mod]

    # Step 4: Build reduced cost graph
    G_reduced = nx.DiGraph()
    for u, v, data in G_split.edges(data=True):
        if u in dist and v in dist:
            new_w = data['weight'] + dist[u] - dist[v]
            G_reduced.add_edge(u, v, weight=new_w, original_weight=data['weight'])

    # Step 5: Remove path1 edges and reverse them
    path_edges = list(zip(path1[:-1], path1[1:]))
    for u, v in path_edges:
        if G_reduced.has_edge(u, v):
            G_reduced.remove_edge(u, v)
        G_reduced.add_edge(v, u, weight=0)

    # Step 6: Second shortest path
    try:
        path2 = nx.shortest_path(G_reduced, s_mod, t_mod, weight='weight')
    except nx.NetworkXNoPath:
        raise nx.NetworkXNoPath("No second disjoint path found")

    # Step 7: Combine edges and eliminate overlaps
    edge_set = set()
    for u, v in zip(path1[:-1], path1[1:]):
        edge_set.add((u, v))
    for u, v in zip(path2[:-1], path2[1:]):
        if (v, u) in edge_set:
            edge_set.remove((v, u))
        else:
            edge_set.add((u, v))

    # Step 8: Build merged graph
    merged = nx.DiGraph()
    for u, v in edge_set:
        merged.add_edge(u, v)

    # Step 9: Extract disjoint paths
    paths = list(nx.edge_disjoint_paths(merged, s_mod, t_mod))
    if len(paths) < 2:
        raise RuntimeError("Failed to extract two disjoint paths")

    # Step 10: Convert back to original node labels
    def unsplit(path):
        result = []
        for node in path:
            node_str = str(node)
            if node_str.endswith('_in') or node_str.endswith('_out'):
                base = int(node_str.split('_')[0])
                if not result or result[-1] != base:
                    result.append(base)
            elif isinstance(node, int):
                if not result or result[-1] != node:
                    result.append(node)
        return result


    return unsplit(paths[0]), unsplit(paths[1])


## Earthquakes Only

### Shortest

we are looking for shortest paths with onit weights on edges

In [ ]:
# Read test data
path = os.path.join("results", "infocom", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        G.edges[edge]["weight"] = 1

    lower_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "earthquakes_shortest.html")
df.to_html(output_path)

### Independent

take the logarithm of the edge fps, as they were independent

In [ ]:
# Read test data
path = os.path.join("results", "infocom", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        intersecting_srlgs = [
            srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
        ]
        G.edges[edge]["weight"] = -np.log10(
            1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
        )

    lower_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "earthquakes_independent.html")
df.to_html(output_path)

## New Data

### Shortest

In [ ]:
# Read test data
path = os.path.join("results", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        G.edges[edge]["weight"] = 1

    lower_bound_ratios = []
    other_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)
        other_bound_ratios.appent(prob-min_prob / base_prob-min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "new_data_shortest.html")
df.to_html(output_path)

### Independent

In [ ]:
# Read test data
path = os.path.join("results", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        intersecting_srlgs = [
            srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
        ]
        G.edges[edge]["weight"] = -np.log10(
            1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
        )

    lower_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "new_data_independent.html")
df.to_html(output_path)

## New Data Edge-Disjoint

### Shortest

In [ ]:
# Read test data
path = os.path.join("results", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        G.edges[edge]["weight"] = 1

    lower_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "edge_disjoint_shortest.html")
df.to_html(output_path)

### Independent

In [ ]:
# Read test data
path = os.path.join("results", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        intersecting_srlgs = [
            srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
        ]
        G.edges[edge]["weight"] = -np.log10(
            1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
        )

    lower_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "edge_disjoint_independent.html")
df.to_html(output_path)

## Full graph 1%

lower bound statistics

In [ ]:
import statistics
method = "min_cutting_prob"
solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    solutions = pickle.load(f)

max_name_len = max(len(name) for name in solutions.keys())
name_col = "Graph"
name_col_width = max(len(name_col), max_name_len)
print(f"{name_col:<{name_col_width}}  {'Min':>8}  {'Max':>8}  {'Avg':>8}  {'Median':>8}")

for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    lower_bouind_ratios = []

    for i, (s,t) in enumerate(solutions[graph_name]["terminals"].keys()):
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)
 
            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)

        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bouind_ratios.append(prob / min_prob)

    minimum = min(lower_bouind_ratios)
    maximum = max(lower_bouind_ratios)
    average = sum(lower_bouind_ratios) / len(lower_bouind_ratios)
    median = statistics.median(lower_bouind_ratios)
    print(f"{graph_name:<{name_col_width}}  {minimum:8.6f}  {maximum:8.6f}  {average:8.6f}  {median:8.6f}")
        

### Compare 1-backup solutions with 2 paths

In [ ]:
default_colors = px.colors.qualitative.Plotly

# Load the results
short_solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    short_solutions = pickle.load(f)
    
# Test parameters
graph_name = "25_interoute_latlon"
G = short_solutions[graph_name]["graph"]
method = "min_cutting_prob"
figures = []

for s, t in short_solutions[graph_name]["terminals"].keys():
    # Our solution
    solution = short_solutions[graph_name]["terminals"][(s, t)][method][2, 1]
    walks = solution["walks"]

    # Disjoint paths
    disjoint_paths = list(nx.node_disjoint_paths(G, s, t, cutoff=2))

    # Get SRLGs intersecting the disjoint paths
    edges_of_disjoint_paths = make_walks_by_edges(G, disjoint_paths)
    disjoint_paths_edge_ids = set(eid for path in edges_of_disjoint_paths for eid in path)
    intersecting_srlgs = [
        srlg for srlg in G.graph['srlgs']
        if set(srlg['edges']) & disjoint_paths_edge_ids
    ]

    # Top SRLGs
    top_srlgs = sorted(intersecting_srlgs, key=lambda s: s['probability'], reverse=True)[:4]
    H = planar_embed(G)
    srlg_traces = get_srlg_traces(H, top_srlgs, dash='dot')
    for idx, trace in enumerate(srlg_traces):
        trace.line.color = default_colors[2 + idx]
        trace.showlegend = (idx == 0)
        trace.name = 'srlgs'

    # Path traces
    traces = get_path_traces(G, disjoint_paths + walks)

    # Disjoint paths
    disjoint_path_traces = traces[:2]
    for i, trace in enumerate(disjoint_path_traces):
        trace.showlegend = (i == 0)
        trace.name = "node disjoint paths" if i == 0 else None
        trace.line.color = default_colors[0]
        if i == 1:
            trace.line.dash = "dash"

    # Our solution
    solution_trace = traces[2:]
    for i, trace in enumerate(solution_trace):
        trace.showlegend = (i == 0)
        trace.name = "our solution" if i == 0 else None
        trace.line.color = default_colors[1]
        if i == 1:
            trace.line.dash = "dash"

    # Base graph
    edge_trace = get_edges_trace(G)
    node_trace = get_nodes_trace(G, label='id')

    # Change the colors of node s and t
    node_colors = ['lightblue'] * len(node_trace.x)
    node_labels = list(node_trace.text)
    s_index = node_labels.index(str(s))
    t_index = node_labels.index(str(t))
    node_colors[s_index] = 'lightpink'
    node_colors[t_index] = 'lightpink'
    node_trace.marker.color = node_colors

    # Combine everything
    traces = disjoint_path_traces + solution_trace + [edge_trace, node_trace] + srlg_traces

    fig = go.Figure(
        data=traces,
        layout=go.Layout(
            margin=dict(b=5, l=5, r=5, t=5),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x"),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )
    )

    fig.update_layout(
        legend=dict(
            x=0.2,
            y=0.99,
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='black',
            borderwidth=1,
            font=dict(size=12),
        ),
    )

    figures.append(fig)

# Dash app
app = Dash()
app.layout = html.Div([
    html.Div([
        html.Div(
            dcc.Graph(figure=figures[i]),
            style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}
        )
        for i in range(len(figures))
    ])
])
app.run(port=8051)


suurballe:
26usa:
fura: 35, 40 41
italy:
fura 4 (10, 16), 14 (9, 10), 

In [ ]:
default_colors = px.colors.qualitative.Plotly

# Load the results
short_solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    short_solutions = pickle.load(f)
    
# Test parameters
graph_name = "25_interoute_latlon"
G = short_solutions[graph_name]["graph"]
method = "min_cutting_prob"
figures = []
# make weights for suurballe
srlgs = G.graph["srlgs"]
for edge in G.edges():
    intersecting_srlgs = [
        srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
    ]
    G.edges[edge]["weight"] = -np.log10(
        1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
    )

for terminal_index, (s, t) in enumerate(short_solutions[graph_name]["terminals"].keys()):
    # Our solution
    solution = short_solutions[graph_name]["terminals"][(s, t)][method][2, 1]
    walks = solution["walks"]
    

    # Disjoint paths
    try:
        disjoint_paths = suurballe_node_disjoint(G, s, t)
    except nx.NetworkXNoPath:
        # edge case for the only node that is not 2-connected
        # instead, take its neighbor
        temp_s = s
        temp_t = t
        if temp_s == 23: temp_s = 17
        if temp_t == 23: temp_t = 17
        disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
        for path in disjoint_paths:
            if path[0] == 17:
                path.insert(0, 23)
            elif path[-1] == 17:
                path.append(23)

    # Get SRLGs intersecting the disjoint paths
    edges_of_disjoint_paths = make_walks_by_edges(G, disjoint_paths)
    disjoint_paths_edge_ids = set(eid for path in edges_of_disjoint_paths for eid in path)
    intersecting_srlgs = [
        srlg for srlg in G.graph['srlgs']
        if set(srlg['edges']) & disjoint_paths_edge_ids
    ]

    # Top SRLGs
    top_srlgs = sorted(intersecting_srlgs, key=lambda s: s['probability'], reverse=True)[:4]
    H = planar_embed(G)
    srlg_traces = get_srlg_traces(H, top_srlgs, dash='dot')
    for idx, trace in enumerate(srlg_traces):
        trace.line.color = default_colors[2 + idx]
        trace.showlegend = (idx == 0)
        trace.name = 'srlgs'

    # Path traces
    disjoint_paths = list(disjoint_paths)
    traces = get_path_traces(G, disjoint_paths + walks)

    # Disjoint paths
    disjoint_path_traces = traces[:2]
    for i, trace in enumerate(disjoint_path_traces):
        trace.showlegend = (i == 0)
        trace.name = "node disjoint paths" if i == 0 else None
        trace.line.color = default_colors[0]
        if i == 1:
            trace.line.dash = "dash"

    # Our solution
    solution_trace = traces[2:]
    for i, trace in enumerate(solution_trace):
        trace.showlegend = (i == 0)
        trace.name = "our solution" if i == 0 else None
        trace.line.color = default_colors[1]
        if i == 1:
            trace.line.dash = "dash"

    # Base graph
    edge_trace = get_edges_trace(G)
    node_trace = get_nodes_trace(G, label='id')

    # Change the colors of node s and t
    node_colors = ['lightblue'] * len(node_trace.x)
    node_labels = list(node_trace.text)
    s_index = node_labels.index(str(s))
    t_index = node_labels.index(str(t))
    node_colors[s_index] = 'lightpink'
    node_colors[t_index] = 'lightpink'
    node_trace.marker.color = node_colors

    # Combine everything
    traces = disjoint_path_traces + solution_trace + [edge_trace, node_trace] + srlg_traces

    fig = go.Figure(
        data=traces,
        layout=go.Layout(
            margin=dict(b=5, l=5, r=5, t=5),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x"),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )
    )

    fig.update_layout(
        legend=dict(
            x=0.2,
            y=0.99,
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='black',
            borderwidth=1,
            font=dict(size=12),
        ),
        title=f"Terminal pair {terminal_index}: s={s}, t={t}",
        title_x=0.5,  # centered
        margin=dict(t=60)
    )

    figures.append(fig)

# Dash app
app = Dash()
app.layout = html.Div([
    html.Div([
        html.Div(
            dcc.Graph(figure=figures[i]),
            style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}
        )
        for i in range(len(figures))
    ])
])
app.run(port=8051)
